In [ ]:
# !pip install lazypredict

In [1]:
import pandas as pd
import numpy as np
import re
# from sklearn.model_selection import train_test_split
# from lazypredict.Supervised import LazyClassifier

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd '/gdrive/MyDrive/Python/LDS0_K273_DangThiPhuongNguyen/Project3/data'

/gdrive/MyDrive/Python/LDS0_K273_DangThiPhuongNguyen/Project3/data


# Đọc Data

In [ ]:
# df = pd.read_csv('data_Foody.csv', index_col=0, encoding='utf-8')
# df.head()

,restaurant,review_text,review_score
0,Good Morning Restaurant,Pizza phong cách Ý. Ngán hải sản nên cả nhà gh...,8.20
1,Hạt Dẻ Trùng Khánh,- Vị trí: chỉ là 1 chiếc xe nho nhỏ nằm ở khu ...,8.00
2,Bột - Healthy & Weight Loss Food - Shop Online,Nay lang thang trên BM thấy đang khuyến mãi xo...,8.20
3,Bánh Giò Chợ Nguyễn Công Trứ,"Quán nằm trong chợ Nguyễn Công Trứ, cũng dễ tì...",8.20
4,Quán Quảng - Bún Quảng,Ngãi heo với heo thiệttttt 🐷🐷🐷\n🤤🤤 bún nem chả...,9.00


In [ ]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39925 entries, 0 to 39924
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   restaurant    39925 non-null  object 
 1   review_text   39925 non-null  object 
 2   review_score  39925 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.2+ MB


In [ ]:
# df.review_text[:10]

0    Pizza phong cách Ý. Ngán hải sản nên cả nhà gh...
1    - Vị trí: chỉ là 1 chiếc xe nho nhỏ nằm ở khu ...
2    Nay lang thang trên BM thấy đang khuyến mãi xo...
3    Quán nằm trong chợ Nguyễn Công Trứ, cũng dễ tì...
4    Ngãi heo với heo thiệttttt 🐷🐷🐷\n🤤🤤 bún nem chả...
5    Quán bờ kè gia đình,giá ăn bình dân,ngon,chủ ...
6    Ghé Mũi né nên thử đồ ăn ở quán. Lẩu cá đục ng...
7    Buffet ở Hải Phòng thì phải chọn Bulgogi!!!!\n...
8    1 năm 2l đi vũng tàu ăn nha mí bạn. Viên to ch...
9    Phong cảnh đẹp, thức uống ngon tại Resort sang...
Name: review_text, dtype: object


# Phân Loại Điểm

In [ ]:
# phân loại điểm đánh giá, nếu lớn hơn 7 là tốt, hơn 5 là trung bình và dưới 5 là tệ
def target_label(x):
  if x >= 7:
    return 2
  elif x >= 5:
    return 1
  else: return 0

In [ ]:
# df['target'] = df.review_score.apply(lambda x: target_label(x))
# df.head()

,restaurant,review_text,review_score,target
0,Good Morning Restaurant,Pizza phong cách Ý. Ngán hải sản nên cả nhà gh...,8.20,2
1,Hạt Dẻ Trùng Khánh,- Vị trí: chỉ là 1 chiếc xe nho nhỏ nằm ở khu ...,8.00,2
2,Bột - Healthy & Weight Loss Food - Shop Online,Nay lang thang trên BM thấy đang khuyến mãi xo...,8.20,2
3,Bánh Giò Chợ Nguyễn Công Trứ,"Quán nằm trong chợ Nguyễn Công Trứ, cũng dễ tì...",8.20,2
4,Quán Quảng - Bún Quảng,Ngãi heo với heo thiệttttt 🐷🐷🐷\n🤤🤤 bún nem chả...,9.00,2


In [ ]:
# df[df['target'] != 2].tail(10)

,restaurant,review_text,review_score,target
39887,Trà Sữa Chiêm Cha,K nghĩ ly trà đào như vậy mà nó thơm mùi đào n...,4.40,0
39888,Ăn Vặt - Trường Kim Đồng - Trần Bình Trọng,Cô chủ xưa giờ nết vẫn vậy nhỉ? Thái độ thật t...,1.00,0
39895,Lotteria - Trần Phú - Hải Phòng,Rất lâu mới quay lại Lotteria*** Phải nói mình...,3.40,0
39897,Hong Cha - Gà Rán & Trà Xí Muội Hàn Quốc,"Có mồi mà thiếu nước, không nhậu bia thì uống ...",3.20,0
39901,Mì Quảng Bé Liên,Quay lại quán lần thứ n. Gía càng ngày có vẻ c...,6.40,1
39915,Lẩu Bò Thảo Nguyên,"Mới đi hôm qua - mùng 5 tết, tầm 7h tối, quán ...",6.20,1
39916,Quán Thảo Hiền - Đặc Sản Miền Trung,Lòng xào nghệ phần nhỏ 40k\nGỏi tai heo phần n...,6.80,1
39917,Long Hương - Cháo Miến Gà Tôm,"Nhà gần, trời lạnh,mặc cái áo khoác có mũ sang...",1.00,0
39919,Bánh Căn Xuân An - Nhà Chung,Nếu so bánh căn quán này với quán cây bơ trên ...,3.60,0
39924,Bạch Dinh Cafe,Mình đi chơi khu Bạch Dinh trong một ngày xong...,5.60,1


# Đổi Ký Hiệu Emoji

In [ ]:
from emoticons import EMOTICONS_EMO

In [ ]:
# đổi emoticon thành chữ
def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = re.sub(u'('+re.escape(emot)+')', "_".join(EMOTICONS_EMO[emot].replace(",","").split()), text)
    return text

# Example
# text = u"Tuy nhiên, lúc gọi món hỏi giá ghẹ tới 700k/kg @@! Lạnh gáy không dám ăn )))) nhưng món khác thì ngon <3"
# convert_emoticons(text)

In [ ]:
# df.review_text = df.review_text.apply(lambda x: str(x).lower())

# Xóa Ký Tự Đặc Biệt

In [ ]:
# xóa ký tự đặc biệt
def remove_special_icons(text):
  for icon in ['\n', ',', '.', '...', '-', ';', '?', '%', '+', '/', '\\', '—', '_', '*', '#', '<', '>', '|', '&', '=', '+', '~', '!', '(', ')', ':', '@', '^']:
    text = text.replace(icon, ' ')
  return text

In [ ]:
# df_ = df
# df_.review_text = df_.review_text.apply(lambda x: remove_special_icons(x))
# df_.head(10)

# Đổi Emoji

In [ ]:
# import emoji dictionary
%cd '/gdrive/MyDrive/Python/LDS0_K273_DangThiPhuongNguyen/Project3/data'
emojicon_dict = {}
emojicon_file = open("emojicon.txt")
for line in emojicon_file:
    key, value = line.split('\t')
    emojicon_dict[key] = value.replace('\n', '')

/gdrive/MyDrive/Python/LDS0_K273_DangThiPhuongNguyen/Project3/data


In [ ]:
# print(emojicon_dict)

In [ ]:
def convert_emoji(text):
    for emoj in emojicon_dict:
        text = text.replace(emoj, "_".join(emojicon_dict[emoj].split())+" ") 
        # thêm space do emoji hay để cạnh nhau
    return text

# Example
# text = u"Macchiato của Koi lun lun là the best 😍😍😍 kem béo béo ngọt ngọt k đâu lẫn đc , uống không cẩn thận là ghiền mất 😭😭😭 Mỗi điểm trừ là giá hơi mắc 😫😫"
# convert_emoji(text)

# Thay Từ Kéo Dài Chữ

In [ ]:
def replace_repeted(text):
    text = re.sub(r'([a-z]|$)\1+', r'\1', text)
    return text

# text = u'thích quán lắmmm nghe đồn nghe mà ăn xong thấy ngon thiệttttt'
# replace_repeted(text)

# Đổi Teencode

In [ ]:
# %cd '/gdrive/MyDrive/Python/LDS0_K273_DangThiPhuongNguyen/Project3/data'

[Errno 107] Transport endpoint is not connected: '/gdrive/MyDrive/Python/LDS0_K273_DangThiPhuongNguyen/Project3/data'
/gdrive/MyDrive/Python/LDS0_K273_DangThiPhuongNguyen/Project3/data


In [ ]:
# mở file teencode
teencode_dict = {}
teencode_file = open("teencode.txt")
for line in teencode_file:
    key, value = line.split('\t')
    teencode_dict[key] = value.replace('\n', '')
print(teencode_dict)

{'ctrai': 'con trai', 'khôg': 'không', 'bme': 'bố mẹ', 'cta': 'chúng ta', 'mih': 'mình', 'mqh': 'mối quan hệ', 'cgai': 'con gái', 'nhữg': 'những', 'mng': 'mọi người', 'svtn': 'sinh viên tình nguyện', 'qtam': 'quan tâm', 'thươg': 'thương', 'qtâm': 'quan tâm', 'chug': 'chung', 'trườg': 'trường', 'thoy': 'thôi', 'đki': 'đăng ký', 'atsm': 'ảo tưởng sức mạnh', 'ạk': 'ạ', 'vch': 'vãi chưởng', 'cùg': 'cùng', 'pn': 'bạn', 'pjt': 'biết', 'thjk': 'thích', 'keke': 'cười', 'ktra': 'kiểm tra', 'nek': 'nè', 'cgái': 'con gái', 'nthe': 'như thế', 'chúg': 'chúng', 'kái': 'cái', 'tìh': 'tình', 'phòg': 'phòng', 'lòg': 'lòng', 'từg': 'từng', 'rằg': 'rằng', 'sốg': 'sống', 'thuj': 'thôi', 'thuơng': 'thương', 'càg': 'càng', 'đky': 'đăng ký', 'bằg': 'bằng', 'sviên': 'sinh viên', 'ák': 'á', 'đág': 'đáng', 'nvay': 'như vậy', 'nhjeu': 'nhiều', 'xg': 'xuống', 'zồi': 'rồi', 'trag': 'trang', 'zữ': 'dữ', 'atrai': 'anh trai', 'kte': 'kinh tế', 'độg': 'động', 'lmht': 'liên minh huyền thoại', 'gắg': 'gắng', 'đzai': 'đẹ

In [ ]:
def convert_teencode(text):
    for word in teencode_dict:
        text = re.sub(u'(^|[.,:;\s])'+re.escape(word)+'(?=[.,\s]|$)', " "+"_".join(teencode_dict[word].split()), text)
    return text

# text = u'người ta ăn gần xong rồi mới tới 2 vc mình, chồng mình nói nv sao người tới trước lại ko đc có trước nv hok buồn trl, còn ko free giữ xe'
# convert_teencode(text)

# Xử Lý Cơ Bản: bỏ in hoa, khoảng trắng thừa và số

In [ ]:
def clean_text(text):
  text = re.sub(r'\d', ' ', text) # bỏ số
  text = re.sub(r'(\s)\1+', ' ', text) # bỏ khoảng trắng liên tục
  return text.strip().lower() # bỏ khoảng trắng đầu cuối và in thường tất cả

# text = '  Đặt hàng lúc 9h15p, giao hàng lúc 9h 35. Mình yêu cầu không để topping và đá: trà sữa matcha được khoảng 180ml.'
# clean_text(text)

In [ ]:
# df_.review_text = df_.review_text.apply(lambda x: convert_emoticons(x))
# df_.review_text = df_.review_text.apply(lambda x: convert_emoji(x))
# df_.review_text = df_.review_text.apply(lambda x: remove_special_icons(x))
# df_.review_text = df_.review_text.apply(lambda x: replace_repeted(x))
# df_.review_text = df_.review_text.apply(lambda x: convert_teencode(x))
# df_.review_text = df_.review_text.apply(lambda x: clean_text(x))
# df_.head(10)

,restaurant,review_text,review_score,target,review_text_wt
0,Good Morning Restaurant,piza phong cách ý ngán hải sản nên cả nhà ghé ...,8.20,2,pizza phong_cách ý ngán hải_sản nên cả nhà ghé...
1,Hạt Dẻ Trùng Khánh,vị trí chỉ là chiếc xe nho nhỏ nằm ở khu ngã s...,8.00,2,vị_trí chỉ là chiếc xe nho_nhỏ nằm ở khu ngã s...
2,Bột - Healthy & Weight Loss Food - Shop Online,nay lang thang trên bm thấy đang khuyến mãi xo...,8.20,2,nay lang_thang trên bm thấy đang khuyến_mãi xo...
3,Bánh Giò Chợ Nguyễn Công Trứ,quán nằm trong chợ nguyễn công trứ cũng dễ tìm...,8.20,2,quán nằm trong chợ nguyễn_công_trứ cũng dễ tìm...
4,Quán Quảng - Bún Quảng,ngãi heo với heo thiệt 🐷🐷🐷 ngon ngon bún nem c...,9.00,2,ngãi heo với heo thiệttttt 🐷_🐷_🐷 ngon ngon bún...
5,Bờ Kè Gia Đình - Hải Sản Bình Dân,quán bờ kè gia đình giá ăn bình dân ngon chủ ...,9.40,2,quán bờ kè gia_đình giá ăn bình_dân ngon chủ q...
6,Vân Anh Quán - Quán Hải Sản,ghé mũi né nên thử đồ ăn ở quán lẩu cá đục ngo...,9.40,2,ghé mũi né nên thử đồ ăn ở quán lẩu cá đục ngo...
7,Bulgogi - Đồ Nướng Than Hoa - Lê Hồng Phong,bufet ở hải phòng thì phải chọn bulgogi đồ ăn ...,7.30,2,buffet ở hải_phòng thì phải chọn bulgogi đồ ăn...
8,Bánh Bạch Tuộc TAKOYAKI - Anh Tuộc,năm l đi vũng tàu ăn nha mí bạn viên to chất l...,8.00,2,năm l đi vũng_tàu ăn nha mí bạn viên to chất l...
9,The L_o_n_g Bar - InterContinental Resort,phong cảnh đẹp thức uống ngon tại resort sang ...,8.80,2,phong_cảnh đẹp thức_uống ngon tại resort sang_...


# Loại Dữ Liệu Không Có Ý Nghĩa

In [ ]:
# print(df_.shape)
# df_ = df_[df_.review_text !=  '']
# print(df_.shape)

(39925, 4)
(39905, 4)


In [ ]:
# df.iloc[15798]

restaurant      Top New Beer Club - Cách Mạng Tháng 8
review_text                                      👆👆👆👆
review_score                                     5.00
target                                              1
Name: 15798, dtype: object

In [ ]:
# df_.drop(15798, inplace=True) # outlier, chỉ chứa emoji không có ý nghĩa

# Tokenize

In [ ]:
# !pip install underthesea

     |████████████████████████████████| 7.6 MB 4.4 MB/s 
     |████████████████████████████████| 235 kB 34.3 MB/s 
     |████████████████████████████████| 743 kB 44.7 MB/s 
     |████████████████████████████████| 581 kB 53.1 MB/s 


In [ ]:
# from underthesea import word_tokenize

In [ ]:
# df_['review_text_wt'] = df_['review_text'].apply(lambda x: word_tokenize(x, format='text'))
# df_[['review_text_wt', 'review_text']].head(10)

,review_text_wt,review_text
0,piza phong_cách ý ngán hải_sản nên cả nhà ghé ...,piza phong cách ý ngán hải sản nên cả nhà ghé ...
1,vị_trí chỉ là chiếc xe nho_nhỏ nằm ở khu ngã s...,vị trí chỉ là chiếc xe nho nhỏ nằm ở khu ngã s...
2,nay lang_thang trên bm thấy đang khuyến_mãi xo...,nay lang thang trên bm thấy đang khuyến mãi xo...
3,quán nằm trong chợ nguyễn_công_trứ cũng dễ tìm...,quán nằm trong chợ nguyễn công trứ cũng dễ tìm...
4,ngãi heo với heo thiệt 🐷_🐷_🐷 ngon ngon bún nem...,ngãi heo với heo thiệt 🐷🐷🐷 ngon ngon bún nem c...
5,quán bờ kè gia_đình giá ăn bình_dân ngon chủ q...,quán bờ kè gia đình giá ăn bình dân ngon chủ ...
6,ghé mũi né nên thử đồ ăn ở quán lẩu cá đục ngo...,ghé mũi né nên thử đồ ăn ở quán lẩu cá đục ngo...
7,bufet ở hải_phòng thì phải chọn bulgogi đồ ăn ...,bufet ở hải phòng thì phải chọn bulgogi đồ ăn ...
8,năm l đi vũng_tàu ăn nha mí bạn viên to chất l...,năm l đi vũng tàu ăn nha mí bạn viên to chất l...
9,phong_cảnh đẹp thức_uống ngon tại resort sang_...,phong cảnh đẹp thức uống ngon tại resort sang ...


# Lọc Từ "Không"

In [ ]:
# import operator
# def find_khong_words(text, ko_list = []):
#   found = re.findall('không+\s\w+', text)
#   for word in found: 
#     ko_list.append(word)
#   return ko_list

In [ ]:
# for id, value in df_.iterrows():
#   ko_list = find_khong_words(value.review_text_wt)

# koWordfreq = {}
# for raw_word in ko_list:
#     word = raw_word.strip()
#     if word not in koWordfreq:
#         koWordfreq[word] = 0 
#     koWordfreq[word] += 1
# koWordfreq = sorted(koWordfreq.items(),key=operator.itemgetter(1),reverse=True)

In [ ]:
# with open('khong_list.txt', 'w') as f:
#     print(koWordfreq, file=f)

In [ ]:
# koWordfreq

[('không có', 11154),
 ('không bị', 4640),
 ('không biết', 4094),
 ('không quá', 4000),
 ('không phải', 3948),
 ('không ngon', 3618),
 ('không được', 3472),
 ('không ăn', 2698),
 ('không thích', 2268),
 ('không thấy', 2198),
 ('không không', 2004),
 ('không mà', 1836),
 ('không hề', 1614),
 ('không cho', 1584),
 ('không bao_giờ', 1446),
 ('không còn', 1446),
 ('không thôi', 1418),
 ('không thì', 1218),
 ('không tô', 1084),
 ('không cần', 922),
 ('không nhưng', 912),
 ('không hiểu', 904),
 ('không mình', 884),
 ('không người', 880),
 ('không quay', 870),
 ('không là', 866),
 ('không phần', 846),
 ('không cái', 834),
 ('không một', 820),
 ('không nên', 802),
 ('không hợp', 774),
 ('không hết', 768),
 ('không ly', 716),
 ('không muốn', 708),
 ('không và', 688),
 ('không bát', 646),
 ('không nhớ', 642),
 ('không nhiều', 616),
 ('không như', 600),
 ('không nói', 576),
 ('không đông', 572),
 ('không đến', 560),
 ('không rõ', 548),
 ('không rẻ', 548),
 ('không ngán', 498),
 ('không giống', 48

Kiểm tra list thì có vài từ hay được sử dụng có thể ảnh hưởng tới sentiment classification có thể thử áp dụng: ['không ngon', 'không được', 'không thích', 'không bao_giờ', 'không hiểu', 'không quay lại', 'không nên', 'không hợp', 'không nhiều', 'không rẻ', 'không đặc_sắc', 'không tươi', 'không kém', 'không hài_lòng']

In [2]:
final_ko_list = ['không có', 'không ngon', 'không được', 'không thích', 'không bao_giờ', 'không hiểu', 'không quay lại', 'không nên', 'không hợp', 'không nhiều',
           'không rẻ', 'không đặc_sắc', 'không tươi', 'không kém', 'không hài_lòng']
text = 'không thích gì cả. đồ ăn thật sự không ngon và không hợp khẩu. không hài_lòng gì cả'
for word in final_ko_list:
  text = re.sub(u'('+re.escape(word)+')', "_".join(word.split()), text)
text

'không_thích gì cả. đồ ăn thật sự không_ngon và không_hợp khẩu. không_hài_lòng gì cả'

In [ ]:
def convert_khong_words(text):
  for word in final_ko_list:
    text = re.sub(u'('+re.escape(word)+')', "_".join(word.split()), text)
  return text

# Lọc Review Tiếng Anh

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect
# detect(df_.review_text[0])

'vi'

In [ ]:
# kiểm tra nhận xét tiếng anh
def check_is_en(text):
  return (detect(text) == 'en')

In [ ]:
# df_.english = df_.review_text.apply(lambda x: 1 if check_is_en(x) else 0)

In [ ]:
# print(df_[df_.english == 1].shape)
# df_[df_.english == 1].head()

(274, 3)


,restaurant,review_text,review_score
28,Riki Riki - Nhà Hàng Gia Đình,very bad food the sashimi wasnt even fresh t...,4.40
38,Thi Thi - Bánh Mì,i ordered their banh mi cha lua with pate one...,8.10
124,Nem 36 - Lê Đại Hành,couple day ago i found this tasty nem nuong cu...,10.00
147,Lytok - Tokbokki & Ăn Vặt Hàn Quốc,after a long time i decided to come back and ...,8.00
232,Phúc Đức Quán - Mì Vằn Thắn,best wonton ever been eating at this restaur...,8.60
